In [8]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
df = pd.read_feather('../mini_codenet/data/split/pretrain_test.ftr')
df.head()

,level_0,index,submission_id,problem_id,language,filename_ext,status,cpu_time,memory,code_size,accuracy,solution,problem_statement
0,951892,15805,s780451974,p02659,Python,py,Accepted,26.0,9812.0,159,None,"from decimal import *\n\nA, B = map(float, inp...",Score : 300 points \n Problem Statement Comput...
1,951893,53848,s571074265,p02659,Python,py,Accepted,26.0,9164.0,88,None,"a, b = input().split()\na = int(a)\nb = int(b[...",Score : 300 points \n Problem Statement Comput...
2,951894,8724,s695103358,p02659,Python,py,Accepted,28.0,9060.0,132,None,"A, B = input().split()\n\nA = int(A)\nB_float ...",Score : 300 points \n Problem Statement Comput...
3,951895,44496,s962640854,p02659,Python,py,Accepted,56.0,61820.0,76,None,"a,b=input().split()\na=int(a)\nb=float(b)\nb*=...",Score : 300 points \n Problem Statement Comput...
4,951896,40612,s914562845,p02659,Python,py,Accepted,92.0,71296.0,190,None,"import decimal\n\na,b = list(input().split())\...",Score : 300 points \n Problem Statement Comput...


In [7]:
print(df.iloc[0]['solution'])
print('=================')
print('=================')
print('=================')
print(df.iloc[0]['problem_statement'])

from decimal import *

A, B = map(float, input().split())

ans = Decimal(str(A)) * Decimal(str(B))
ans = str(ans)
i = ans.find('.')
ans = ans[:i]


print(ans)

Score : 300 points 
 Problem Statement Compute A \times B , truncate its fractional part, and print the result as an integer. 
 Constraints 
 0 \leq A \leq 10^{15} 
 0 \leq B &lt; 10 
 A is an integer. 
 B is a number with two digits after the decimal point. 
 Input Input is given from Standard Input in the following format: 
``` A B 
```
 Output Print the answer as an integer. 
 Sample Input 1 ```198 1.10
```
 Sample Output 1 ```217
```
 We have 198 \times 1.10 = 217.8 . After truncating the fractional part, we have the answer: 217 . 
 Sample Input 2 ```1 0.01
```
 Sample Output 2 ```0
```
 Sample Input 3 ```1000000000000000 9.99
```
 Sample Output 3 ```9990000000000000
```


In [10]:
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModelForCausalLM.from_pretrained("gpt2-large")
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load('./model/cnds_model3.pt'))
model = model.to('cuda:0')
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50258, bias=False)
)

In [11]:
def generate(text_in, tok_in, mod_in):
    tok_text = tok_in(text_in, return_tensors='pt').to('cuda:0')
    gen_text = mod_in.generate(**tok_text, max_new_tokens=512)
    dec_text = tok_in.decode(gen_text[0], skip_special_tokens=True)
    return dec_text

In [12]:
p = df.iloc[0]['problem_statement']
text_in = f'Python::{p}::'

print(generate(text_in, tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Python::Score : 300 points 
 Problem Statement Compute A \times B, truncate its fractional part, and print the result as an integer. 
 Constraints 
 0 \leq A \leq 10^{15} 
 0 \leq B &lt; 10 
 A is an integer. 
 B is a number with two digits after the decimal point. 
 Input Input is given from Standard Input in the following format: 
``` A B 
```
 Output Print the answer as an integer. 
 Sample Input 1 ```198 1.10
```
 Sample Output 1 ```217
```
 We have 198 \times 1.10 = 217.8. After truncating the fractional part, we have the answer: 217. 
 Sample Input 2 ```1 0.01
```
 Sample Output 2 ```0
```
 Sample Input 3 ```1000000000000000 9.99
```
 Sample Output 3 ```9990000000000000
```::a

 Sample Input 4 ```1000000000000000 9.99
```
 Sample Output 4 ```9990000000000000
``::a
 Sample Input 5 ```1000000000000000 9.99
```
 Sample Output 5 ```9990000000000000
``::a
 Sample Input 6 ```1000000000000000 9.99
```
 Sample Output 6 ```9990000000000000
``::a
 Sample Input 7 ```1000000000000000 9.99
``